In [12]:
import pandas as pd

df = pd.read_csv("C:/Users/VIGNEESH GPL/BigData/data2/student_performance_preprocessed.csv")
raw_df = df.copy()

# Define Columns
numeric_cols = ["studyhours", "attendance", "age", "examscore"]
categorical_cols = [
    "resources", "extracurricular", "motivation", "internet",
    "gender", "learningstyle", "onlinecourses", "discussions",
    "assignmentcompletion", "edutech", "stresslevel", "finalgrade"
]

# Handle Missing Values
df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.mean()))
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(x.mode()[0]))

print("Missing values handled:")
print(df.isna().sum())

Missing values handled:
studyhours              0
attendance              0
resources               0
extracurricular         0
motivation              0
internet                0
gender                  0
age                     0
learningstyle           0
onlinecourses           0
discussions             0
assignmentcompletion    0
examscore               0
edutech                 0
stresslevel             0
finalgrade              0
dtype: int64


In [13]:
# Convert Data Types & Optimize Memory

# Convert numeric columns safely
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Convert categorical columns and strip spaces
for col in categorical_cols:
    df[col] = df[col].astype(str).str.strip().astype("category")

# Show final data types
print(df.dtypes)

studyhours                 int64
attendance                 int64
resources               category
extracurricular         category
motivation              category
internet                category
gender                  category
age                        int64
learningstyle           category
onlinecourses           category
discussions             category
assignmentcompletion    category
examscore                  int64
edutech                 category
stresslevel             category
finalgrade              category
dtype: object


In [14]:
# --- After missing values and duplicate handling ---

# Remove 'examscore' (it's highly correlated with finalgrade)
if 'examscore' in df.columns:
    df.drop(columns=['examscore'], inplace=True)
    print("Removed examscore")


Removed examscore


In [15]:
# Remove Duplicates
rows_before = df.shape[0]
df.drop_duplicates(inplace=True)
rows_after = df.shape[0]
print(f"\nDuplicates removed: {rows_before - rows_after}")
df.reset_index(drop=True, inplace=True)


Duplicates removed: 2168


In [16]:
# Outlier Removal (IQR Method)
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

numeric_outlier_cols = ["studyhours", "age"]

rows_before_outlier = len(df)
for col in numeric_outlier_cols:
    df = remove_outliers_iqr(df, col)
df.reset_index(drop=True, inplace=True)
rows_after_outlier = len(df)

print(f"\nOutliers removed: {rows_before_outlier - rows_after_outlier} rows")
print(f"Remaining rows: {rows_after_outlier}")


Outliers removed: 45 rows
Remaining rows: 11790


In [17]:
# Final Summary
print("Shape:", df.shape)
print("Missing Values:", df.isnull().sum().sum())
print("Duplicates:", df.duplicated().sum())

Shape: (11790, 15)
Missing Values: 0
Duplicates: 0


In [20]:
# Cleaned Dataset
output_path = "C:/Users/VIGNEESH GPL/BigData/data2/student_performance_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"\nCleaned dataset saved to:\n{output_path}")


Cleaned dataset saved to:
C:/Users/VIGNEESH GPL/BigData/data2/student_performance_cleaned.csv


In [21]:
invalid_rows = df[(df['studyhours'] < 0) | (df['attendance'] > 100)]
print(f"\nInvalid logical rows found: {len(invalid_rows)}")


Invalid logical rows found: 0
